In [ ]:
%load_ext autoreload
%autoreload 2
from module import *

'''Experiment is xz_13_14 '''
exp = 'xz_13_14'
'''groupby can be {none,13-14-reduced}:'''
# groupby = 'none'
groupby = '13-14-reduced'
resolution = 10000000
RC_threshold = 0

positionlist,chromlist,border = structure_genome(resolution)
X,index0_list,index1_list,index2_list,index3_list = load_data(exp,resolution,positionlist)
X_df,X_red_df,X,zero_col = load_RCdf(X,exp,chromlist,positionlist,groupby,RC_threshold,index0_list,index1_list,index2_list,index3_list)

In [ ]:
'''
Estimate the percentage of nnz elements in XX
'''
print 'The percentage of nnz entries in X is:', np.count_nonzero(X)*1.0/(X.shape[0]*X.shape[1])*100,'%'
print 'The dimension of X are:', X.shape

In [ ]:
X_filled_softimpute = impute(X,zero_col)
X_filled_normalized_df,X_filled_df = normalize(X_filled_softimpute,X_red_df) 

In [ ]:
'''Hierarchical cluster'''
import matplotlib.pyplot as plt
import seaborn as sns

method = 'average'
metric = 'euclidean'

# xticklabels=[]
g = sns.clustermap(X_filled_normalized_df,method=method,metric=metric,col_cluster=False,\
                  linewidths=0.0,cmap='Greys')

plt.title('Exp'+str(exp)+'res10M'+'_'+'GroupBy'+str(groupby)+'_'+'Threshold'+str(RC_threshold))

plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.setp(g.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)

plt.setp(g.ax_heatmap.set_xticks(border))

for i in border:
    plt.setp(g.ax_heatmap.axvline(i,c="r",lw=0.1))
plt.show()

g.savefig('ExpXZ1314_res10M'+'_'+'GroupBy'+str(groupby)+'_'+'Threshold'+str(RC_threshold)+'.png')

In [ ]:
shannon_df_xz1314,shannon_df_xz13,shannon_df_xz14 = shannonDF(X_filled_df)
totalRC_df_xz1314,totalRC_df_xz13,totalRC_df_xz14 = readcountDF(X_filled_df) 
delta_df_xz1314,delta_df_xz13,delta_df_xz14 = NNdistDF(X_filled_df,shannon_df_xz1314,shannon_df_xz13,shannon_df_xz14,totalRC_df_xz1314,totalRC_df_xz13,totalRC_df_xz14)  
fluctuation_df_xz1314,fluctuation_df_xz13,fluctuation_df_xz14 = fluctuationDF(X_filled_df,X_filled_softimpute)

In [ ]:
local_fluctuation_df = local_fluctuationsDF(X_filled_df)

In [ ]:
'''
Distribution of fluctuations
'''
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(1)
# arr = delta_df_xz13.values
# g1 = sns.distplot(arr[arr>0],label='xz13')
# arr = delta_df_xz14.values
# g2 = sns.distplot(arr[arr>0],label='xz14')
arr = local_fluctuation_df.values
g3 = sns.distplot(arr[arr>0],label='local')
arr = delta_df_xz1314.values
g4 = sns.distplot(arr[arr>0],label='xz1314')
plt.legend()
plt.title('Distributions of local fluctuations and threshold:'+str(RC_threshold))

# plt.figure(2)
# arr = fluctuation_df_xz13.values
# g1 = sns.distplot(arr[arr>0],label='xz13')
# arr = fluctuation_df_xz14.values
# g2 = sns.distplot(arr[arr>0],label='xz14')
# arr = fluctuation_df_xz1314.values
# g3 = sns.distplot(arr[arr>0],label='xz1314')
# plt.legend()
# plt.title('Distributions of fluctuations from the mean and threshold:'+str(RC_threshold))

plt.show()

In [ ]:
'''
Spatial vs Genomic correlation.
The plot suggests that there is a short scale distance at which cells typically are more dissimilar increasing their
distance, then after a threshold distance they start to be more alike, as in the linear sequence: 
0,1,2,0,1,2,0,1,2....
Like there was a periodic pattern in the functinal space, which could correspond to the size of the clones
'''
def spatialDist(row0,col0,row1,col1):
#     return np.sqrt( abs(ord(row0)-ord(row1))**2 + abs(int(col0)-int(col1))**2 )
    return abs(ord(row0)-ord(row1)) + abs(int(col0)-int(col1))

def cosineDist(df,ind1a,ind2a,ind1b,ind2b):
    return distance.cosine(df.ix[ind1a,ind2a],df.ix[ind1b,ind2b])

ind1list = ['A','B','C','D','E','F','G','H','I','J']
ind2list = ['1','2','3','4','5','6','7']
df = X_filled_df
x = []
y = []
for ind1a in ind1list:
    for ind1b in ind1list:
        for ind2a in ind2list:
            for ind2b in ind2list:
                if ((ind1a,ind2a) in df.index) and ((ind1b,ind2b) in df.index):
                    x.append(spatialDist(ind1a,ind2a,ind1b,ind2b))
                    y.append(cosineDist(df,ind1a,ind2a,ind1b,ind2b))
import matplotlib.pyplot as plt
import seaborn as sns
sns.boxplot(np.asarray(x),np.asarray(y))
plt.title('Functional vs spatial distance')
# plt.ylim(0, 0.3)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

g = sns.heatmap(totalRC_df_xz1314,square=False, annot=True, cmap="Blues", cbar=True)
plt.title('RC  for XZ13+14 and threshold:'+str(RC_threshold))
plt.show()

g = sns.heatmap(delta_df_xz1314,square=False, annot=False, cmap="Blues", cbar=True)
plt.title('Cosine distance  for XZ13+14 and threshold:'+str(RC_threshold))
plt.show()

g = sns.heatmap(fluctuation_df_xz1314, square=False, annot=False, cmap="Blues", cbar=True)
plt.title('Fluctuations from the mean  for XZ13+14 and threshold:'+str(RC_threshold))
plt.show()

plt.figure(1)
arr = delta_df_xz1314.values
g1 = sns.distplot(arr[arr>0],label='xz13+14')
plt.legend()
plt.title('Distributions of local fluctuations for XZ13+14 and threshold:'+str(RC_threshold))
plt.figure(2)
arr = fluctuation_df_xz1314.values
g1 = sns.distplot(arr[arr>0],label='xz13+14')
plt.legend()
plt.title('Distributions of fluctuations from the mean for XZ13+14 and threshold:'+str(RC_threshold))
plt.show()

In [ ]:
y1 = np.asarray(delta_df_xz13)[np.asarray(delta_df_xz13)>0]
y2 = np.asarray(delta_df_xz13)[np.asarray(delta_df_xz14)>0]
y3 = np.asarray(delta_df_xz1314)[np.asarray(delta_df_xz1314)>0]
y4 = np.asarray(local_fluctuation_df)[np.asarray(local_fluctuation_df)>0]
sns.boxplot(data=[y1,y2,y3])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(1)
g1 = sns.heatmap(abs(totalRC_df_xz13),square=False, annot=True, cmap="Blues", cbar=True)
plt.title('RC for xz13 and threshold:'+str(RC_threshold))
plt.figure(2)
g2 = sns.heatmap(abs(totalRC_df_xz14),square=False, annot=True, cmap="Blues", cbar=True)
plt.title('RC for xz14 and threshold:'+str(RC_threshold))
plt.show()

plt.figure(1)
g1 = sns.heatmap(abs(delta_df_xz13),square=False, annot=False, cmap="Blues", cbar=True)
plt.title('Cosine distance for xz13 and threshold:'+str(RC_threshold))
plt.figure(2)
g2 = sns.heatmap(abs(delta_df_xz14),square=False, annot=False, cmap="Blues", cbar=True)
plt.title('Cosine distance for xz14 and threshold:'+str(RC_threshold))
plt.show()

plt.figure(1)
g1 = sns.heatmap(fluctuation_df_xz13,square=False, annot=False, cmap="Blues", cbar=True)
plt.title('Fluctuations from the average for XZ13 and threshold:'+str(RC_threshold))
plt.figure(2)
g2 = sns.heatmap(fluctuation_df_xz14,square=False, annot=False, cmap="Blues", cbar=True)
plt.title('Fluctuations from the average for XZ14 and threshold:'+str(RC_threshold))
plt.show()

In [ ]:
'''
Fluctuations between co-localized regions, measured with cosine distance
'''
import matplotlib.pyplot as plt
import seaborn as sns
# # pd.set_option('chop_threshold',0.0001)
# # print local_fluctuation_df
# # Use abs() to account for very small negative values
g = sns.heatmap(abs(local_fluctuation_df),square=False, annot=False, cmap="Blues", cbar=True)
plt.title('Local distances between colocalized fragments and threshold:'+str(RC_threshold))
plt.show()

In [ ]:
'''
Spatial vs Genomic correlation.
The plot suggests that there is a short scale distance at which cells typically are more dissimilar increasing their
distance, then after a threshold distance they start to be more alike, as in the linear sequence: 
0,1,2,0,1,2,0,1,2....
Like there was a periodic pattern in the functinal space, which could correspond to the size of the clones
'''
def spatialDist(row0,col0,row1,col1):
    return np.sqrt(abs(ord(row0)-ord(row1))**2+abs(int(col0)-int(col1))**2)
def cosineDist(df,ind0,ind1a,ind2a,ind1b,ind2b):
    return distance.cosine(df.ix[ind0,ind1a,ind2a],df.ix[ind0,ind1b,ind2b])
ind0list = ['xz13','xz14']
ind1list = ['A','B','C','D','E','F','G','H','I','J']
ind2list = ['1','2','3','4','5','6','7']
df = X_filled_df
x = []
y = []
for ind0 in ind0list:
    for ind1a in ind1list:
        for ind1b in ind1list:
            for ind2a in ind2list:
                for ind2b in ind2list:
                    if ((ind0,ind1a,ind2a) in df.index) and ((ind0,ind1b,ind2b) in df.index):
                        x.append(spatialDist(ind1a,ind2a,ind1b,ind2b))
                        y.append(cosineDist(df,ind0,ind1a,ind2a,ind1b,ind2b))
import matplotlib.pyplot as plt
import seaborn as sns
# plt.scatter(np.asarray(x),np.asarray(y))
sns.regplot(np.asarray(x),np.asarray(y),x_estimator=np.mean,fit_reg=False)
plt.title('Functional vs spatial distance')
plt.show()

In [ ]:
'''Distance between regions'''
from scipy.spatial import distance

dist = 'canberra'
dm = distance.pdist(X_filled_normalized,dist)
# dm = distance.pdist(X_filled_normalized, lambda u, v: np.sqrt(np.dot(u,v)).sum())
Y = distance.squareform( dm )

import seaborn as sns

mask = np.zeros_like(Y)
mask[np.diag_indices_from(mask)] = True
with sns.axes_style("white"):
    g = sns.heatmap(Y, mask=mask, square=True, annot=False, cmap="Blues")
    g.set(xticklabels=[])
    g.set(yticklabels=[])
    
plt.show()

In [ ]:
'''
See the sparsity structure of X
'''
import matplotlib.pyplot as plt

plt.figure(1)
plt.spy(X[:,:100])
plt.show()

In [ ]:
'''Heatmap after imputation'''

import seaborn as sns

g1 = sns.heatmap(X_red_df,square=False, annot=False, cmap="Blues", cbar=False)
plt.title('Data Matrix')
g1.set(xticklabels=[])
plt.show()
# g1.savefig('data_matrix.pdf')

g2 = sns.heatmap(X_filled_softimpute, square=False, annot=False, cmap="Blues", cbar=False)
plt.title('SoftImpute matrix completion')
g2.set(xticklabels=[])
plt.show()
# g2.savefig('imputed_data_matrix.pdf')

In [ ]:
'''Heatmap after normalization'''

import seaborn as sns

plt.figure(1)
g1 = sns.heatmap(X_filled_normalized_df,square=False, annot=False, cmap="Blues", cbar=False)
plt.title('Imputed matrix after normalization')
g1.set(xticklabels=[])

plt.figure(2)
g2 = sns.heatmap(X_filled_softimpute, square=False, annot=False, cmap="Blues", cbar=False)
plt.title('SoftImpute matrix completion')
g2.set(xticklabels=[])

plt.show()

In [ ]:
'''SVD of the data matrix'''

U,s,V = np.linalg.svd(X_filled_normalized)

import matplotlib.pyplot as plt
for ind in range(3):
    plt.figure(ind)
    plt.bar(range(V.shape[1]),abs(V[:,ind]))
        
plt.show()

In [ ]:
# '''Shannon entropy'''
# from scipy.stats import entropy

# shannon = []
# Z_13 = np.zeros([11,8])
# Z_14 = np.zeros([11,8])
# for i in X_filled_normalized_df.index:
#     shannon.append( entropy(X_filled_normalized_df.ix[i]) )
# for i in range(len(X_filled_normalized_df.index.values)):
#     if X_filled_normalized_df.index.values[i][0]=='xz13':
#         xx = X_filled_normalized_df.index.values[i][1].lower()
#         Z_13[ord(xx) - 96,int(X_filled_normalized_df.index.values[i][2])] = shannon[i]
#     if X_filled_normalized_df.index.values[i][0]=='xz14':
#         xx = X_filled_normalized_df.index.values[i][1].lower()
#         Z_14[ord(xx) - 96,int(X_filled_normalized_df.index.values[i][2])] = shannon[i]

# # import matplotlib
# # import numpy as np
# # import matplotlib.pyplot as plt

# # plt.figure(1)
# # CS = plt.contour(Z_13)
# # plt.clabel(CS, inline=1, fontsize=10)
# # plt.title('XZ13')

# # plt.figure(2)
# # CS = plt.contour(Z_14)
# # plt.clabel(CS, inline=1, fontsize=10)
# # plt.title('XZ14')

# # plt.show()

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.offline import plot
import plotly.plotly as py
import plotly.graph_objs as go
init_notebook_mode()
'''
Plot results
'''
# py.sign_in('garner1','5dt8q3q49m')
data_xz13 = [
    go.Contour(
        z=shannon_df_xz13.as_matrix(),
        x=columns,
        y=index
    )
]
data_xz14 = [
    go.Contour(
        z=shannon_df_xz14.as_matrix(),
        x=columns,
        y=index
    )
]
plot(data_xz13, filename='contour for xz13b')
plot(data_xz14, filename='contour for xz14')

# data_xz13 = [
#     go.Heatmap(
#         z=shannon_df_xz13.as_matrix(),
#         x=columns,
#         y=index
#     )
# ]
# data_xz14 = [
#     go.Heatmap(
#         z=shannon_df_xz14.as_matrix(),
#         x=columns,
#         y=index
#     )
# ]
# plot(data_xz13, filename='heatmap for xz13')
# plot(data_xz14, filename='heatmap for xz14')

# data_xz13 = [
#     go.Heatmap(
#         z=totalRC_df_xz13.as_matrix(),
#         x=columns,
#         y=index
#     )
# ]
# data_xz14 = [
#     go.Heatmap(
#         z=totalRC_df_xz14.as_matrix(),
#         x=columns,
#         y=index
#     )
# ]
# plot(data_xz13, filename='RC for xz13')
# plot(data_xz14, filename='RC for xz14')